# 基于Mindspore构造Avg Pooling

池化操作（也称为子采样或下采样）主要为了降低每个特征图的维数，可以减少参数矩阵的尺寸，从而减少最后输出的数量，但保留了最重要的信息。池化可以有不同的类型：MaxPooling、AveragePooling、SumPooling等。
若核为2， 步长为1.则在每一个2*2的区域内，选取平均值作为输出。若我们需要的对象偏向于整体特性，防止丢失太多的高维信息则可选择平均池化。


## 使用示例

In [ ]:
class mindspore.ops.AvgPool(kernel_size=1, strides=1, pad_mode="valid", data_format="NCHW")

In [1]:
from mindspore.common.tensor import Tensor
import mindspore.nn as nn
import numpy as np
import mindspore.common.dtype as mstype

pool = nn.AvgPool2d(kernel_size=3, stride=1)
x = Tensor(np.random.randint(0, 10, [1, 2, 4, 4]), mstype.float32)
output = pool(x)
print(output.shape)



(1, 2, 2, 2)


### 常用参数说明
``` python
in_channels (int) - Conv2d层输入Tensor的空间维度。

out_channels (int) - Conv2d层输出Tensor的空间维度。

kernel_size (Union[int, tuple[int]]) - 指定二维卷积核的高度和宽度。数据类型为整型或两个整型的tuple。一个整数表示卷积核的高度和宽度均为该值。两个整数的tuple分别表示卷积核的高度和宽度。

stride (Union[int, tuple[int]]) - 二维卷积核的移动步长。数据类型为整型或两个整型的tuple。一个整数表示在高度和宽度方向的移动步长均为该值。两个整数的tuple分别表示在高度和宽度方向的移动步长。默认值：1。

pad_mode (str) - 指定填充模式。可选值为”same”、”valid”、”pad”。默认值：”same”。

padding (Union[int, tuple[int]]) - 输入的高度和宽度方向上填充的数量。数据类型为int或包含4个整数的tuple。如果 padding 是一个整数，那么上、下、左、右的填充都等于 padding 。如果 padding 是一个有4个整数的tuple，那么上、下、左、右的填充分别等于 padding[0] 、 padding[1] 、 padding[2] 和 padding[3] 。值应该要大于等于0，默认值：0。
```

# 自行实现Mindspore Avg Pooling API

## 导入所需包

In [2]:
import mindspore.nn as nn
import numpy as np
import mindspore.common.dtype as mstype

from mindspore.ops import operations as P
from mindspore._checkparam import Rel, Validator as validator
from mindspore.ops.primitive import constexpr
from mindspore.common.tensor import Tensor
import mindspore.context as context

from mindspore.nn.cell import Cell

## 使用Mindspore官方定义的基类

In [3]:
class _PoolNd(Cell):
    """N-D  AvgPool"""

    def __init__(self, kernel_size, stride, pad_mode, data_format="NCHW"):
        """Initialize _PoolNd."""
        super(_PoolNd, self).__init__()
        validator.check_value_type('pad_mode', pad_mode, [str], self.cls_name)
        self.pad_mode = validator.check_string(pad_mode.upper(), ['VALID', 'SAME'], 'pad_mode', self.cls_name)
        self.format = validator.check_string(data_format, ['NCHW', 'NHWC'], 'format', self.cls_name)
        if context.get_context("device_target") != "GPU" and self.format == "NHWC":
            raise ValueError(f"For '{self.cls_name}, the 'NHWC' format only support in GPU target, but got device "
                             f"target {context.get_context('device_target')}.")

        def _check_int_or_tuple(arg_name, arg_value):
            validator.check_value_type(arg_name, arg_value, [int, tuple], self.cls_name)
            error_msg = f"For '{self.cls_name}', the '{arg_name}' must be an positive int number or " \
                        f"a tuple of two positive int numbers, but got {arg_value}"
            if isinstance(arg_value, int):
                if arg_value <= 0:
                    raise ValueError(error_msg)
            elif len(arg_value) == 2:
                for item in arg_value:
                    if isinstance(item, int) and item > 0:
                        continue
                    raise ValueError(error_msg)
            else:
                raise ValueError(error_msg)
            return arg_value

        self.kernel_size = _check_int_or_tuple('kernel_size', kernel_size)
        self.stride = _check_int_or_tuple('stride', stride)

    def construct(self, *inputs):
        pass

    def extend_repr(self):
        return 'kernel_size={kernel_size}, stride={stride}, pad_mode={pad_mode}'.format(**self.__dict__)


@constexpr
def _shape_check(in_shape, prim_name=None):
    msg_prefix = f"For '{prim_name}', the" if prim_name else "The"
    if len(in_shape) != 3:
        raise ValueError(f"{msg_prefix} input must has 3 dim, but got {len(in_shape)}")


## 实现二维AvgPool

In [4]:
class AvgPool2d(_PoolNd):
    

    def __init__(self,
                 kernel_size=1,
                 stride=1,
                 pad_mode="valid",
                 data_format="NCHW"):
        """Initialize AvgPool2d."""
        super(AvgPool2d, self).__init__(kernel_size, stride, pad_mode, data_format)
        self.avg_pool = P.AvgPool(kernel_size=self.kernel_size,
                                  strides=self.stride,
                                  pad_mode=self.pad_mode,
                                  data_format=self.format)

    def construct(self, x):
        return self.avg_pool(x)
pool = nn.AvgPool2d(kernel_size=3, stride=1)
x = Tensor(np.random.randint(0, 10, [1, 2, 4, 4]), mstype.float32)
output = pool(x)
print(output)
print(output.shape)

[[[[5.2226562 5.109375 ]
   [3.8886719 4.109375 ]]

  [[3.6660156 4.6640625]
   [5.4414062 5.109375 ]]]]
(1, 2, 2, 2)
